Problem statement
1 (a nice linear space)
GPS - solve for x,y,z,t given measured t's from many sources.

2. (with some constraints)

Spaceship, solve for quaternion and position, given many directions of stars.

3. (more constraints)

Articulated spaceship?